In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio

# Change working directory to project root
os.chdir('../../')

In [2]:
# Create Wis3D
from hmr4d.utils.wis3d_utils import make_wis3d, get_gradient_colors, get_const_colors
wis3d = make_wis3d(name="triangulation", time_postfix=False)

### AMASS: orthogonal + control 1 view

In [3]:
# Read from RICH dataset. To form an example data.
from hmr4d.dataset.amass.amass_motion_multiview import Dataset

cfg_ = {"root": "inputs/amass/smplh_joints3d.pth", "is_pinhole": False, "is_uniform": False}
dataloader = torch.utils.data.DataLoader(Dataset(**cfg_), batch_size=4, shuffle=False, num_workers=0)
dataloader_iter = iter(dataloader)
batch = next(dataloader_iter)
print(batch.keys())

for k in batch.keys():
    print(f"{k}: {batch[k].shape}")

dict_keys(['length', 'bbxs_lurb', 'bi01_motion2ds', 'gt_w_motion', 'Ts_w2c', 'is_pinhole', 'Ks'])
length: torch.Size([4])
bbxs_lurb: torch.Size([4, 5, 4])
bi01_motion2ds: torch.Size([4, 5, 120, 22, 2])
gt_w_motion: torch.Size([4, 120, 22, 3])
Ts_w2c: torch.Size([4, 5, 4, 4])
is_pinhole: torch.Size([4])
Ks: torch.Size([4, 5, 3, 3])


In [23]:
from einops import rearrange, repeat, einsum
from hmr4d.utils.geo_transform import apply_T_on_points, project_p2d

Ts_w2c = batch["Ts_w2c"]  # (B, V=5, 4, 4)
V = Ts_w2c.shape[1]

# Get c_p2d (B, V, N, 2)
w_p3d = batch["gt_w_motion"][:, 0, :, :]  # (B, N=23, 3)
c_p3d = rearrange(
    apply_T_on_points(repeat(w_p3d, "b n c -> (b v) n c", v=V), rearrange(Ts_w2c, "b v c d -> (b v) c d")),
    "(b v) n c -> b v n c",
    v=V,
)  # (B, V=5, N, 3)
c_p2d = c_p3d[:, :, :, :2]  # simple orthognal projection (B, V, N, 2)

# from hmr4d.utils.geo_transform import triangulate_point_ortho
# w_p3d_  = triangulate_point_ortho(Ts_w2c, c_p2d)
# print(f"Error: {torch.norm(w_p3d - w_p3d_, dim=-1).mean()}")  # -> 4.408202514127879e-08

In [37]:
from hmr4d.utils.geo.triangulation import triangulate_ortho_c1v
w_p3d_triangulated = triangulate_ortho_c1v(Ts_w2c, c_p2d, controled_viewid=0)  # B, N, 3
print(f"Error: {torch.norm(w_p3d - w_p3d_, dim=-1).mean()}")  # -> 4.408202514127879e-08

b_vis = 3
# Vis prediction
colors = get_const_colors("red", (w_p3d.shape[1],))
wis3d.add_point_cloud(w_p3d_triangulated[b_vis], colors=colors, name="w_p3d_triangulated")

# Vis gt
colors = get_const_colors("green", (w_p3d.shape[1],))
wis3d.add_point_cloud(w_p3d[b_vis], colors=colors, name="w_p3d")

Error: 4.408202514127879e-08


### RICH: perspective triangulation

In [3]:
# Read from RICH dataset. To form an example data.
from hmr4d.dataset.rich.rich_motion_multiview import Dataset
dataset = Dataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=False, num_workers=0)
dataloader_iter = iter(dataloader)
batch = next(dataloader_iter)
print(batch.keys())

dict_keys(['length', 'gt_w_motion', 'Ts_w2c', 'Ks', 'bbxs_lurb', 'bi01_motion2ds'])


In [4]:
from hmr4d.utils.wis3d_utils import add_one_joints22_as_lines

# Use the first frame in the batch to examine the triangulation.
gt_w_p3d = batch['gt_w_motion'][0, 0, :, :]  # (22, 3)
colors = get_const_colors("green", (gt_w_p3d.shape[0],))
wis3d.add_point_cloud(gt_w_p3d, colors=colors, name='gt_w_p3d')

In [6]:
# Compute the projected points in each view.
from hmr4d.utils.geo_transform import apply_T_on_points, triangulate_point

c_p2d = []  # observed
for v in range(3):
    gt_w_p3d = batch['gt_w_motion'][:, 0, :, :]  # (B=1, 22, 3)
    gt_c_p3d = apply_T_on_points(gt_w_p3d, batch['Ts_w2c'][:, v])  # (B, 22, 3)
    gt_c_p2d = gt_c_p3d[..., :2] / gt_c_p3d[..., [2]]  # (B, 22, 2)
    c_p2d.append(gt_c_p2d)
c_p2d = torch.stack(c_p2d, dim=1)  # (B, V, 22, 2)

# Add noise to c_p2d
noise = torch.randn_like(c_p2d) * 0.01
c_p2d += noise

# Visualize
# for v in range(3):
#     c_p2d_ = c_p2d[0, v, :, :]  # (22, 2)
#     c_p3d_ = F.pad(c_p2d_, (0, 1), value=1)  # (22, 3)
#     add_one_joints22_as_lines(c_p3d_, wis3d, name=f'gt_c_p2d_{v}')

# Triangulate
w_p3d = triangulate_point(batch["Ts_w2c"], c_p2d)  # (B, N, 3)
error = torch.linalg.norm(w_p3d - batch["gt_w_motion"][:, 0, :, :3], dim=2).mean()
print(error)

# Add predicted point to wis3d
colors = get_const_colors("red", (w_p3d.shape[1],))
wis3d.add_point_cloud(w_p3d[0], colors=colors, name="w_p3d")

tensor(0.0334)
